# Transfer Learning with TensorFlow

Classical methods showed poor performance for image and text classification problems. , the model which based on deep learing used to solve these problems. There are very much popular models which test and open source that they are avaible for transfer learning.

The models such as ResNet are prebuilt and very complicated structures. TensorFlow Hub proposed a variety of ML models freely. You can use these model writting a single line of code. 

In this tutorial, I am going to show how to use models pretrained in TensorFlow Hub. If you want to install Tensorflow Hub, you can use 'pip install --upgrade tensorflow_hub' command. After installing, first of all, to use I am going to import this library .

In [ ]:
import tensorflow_hub as hub

Let's show how to use a pretrained text embedding model in tensorFlow Hub.

In [ ]:
model = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128/2")

Token in this model based text embedding trained on English Google News 200B corpus. Text embedding is a multidimensional vector of numeric representation. Note that you can onlu load and use it to get a result with your own data. 

You can choose the model in [TensorFlow Hub](https://tfhub.dev/) according to your the dataset. There are very much model for image, text, video, and audio datasets.

## Image Classification by Transfer Learning

In this section, I am going to talk about how to analyze with transfer learning for classification problem. To show this, let me use flower photos dataset which provided by google. 
First of all, let's import libraries.

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pylab as plt

## Loading the Dataset

I am going to load the dataset using get_file() method.

In [ ]:
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

By default, you can see this dataset in .keras file such as /root/.keras/datasets/flower_photos. 

## Preprocessing the Dataset

Now that I am going to define some hyperparameters such as pixel values and batch size.

In [ ]:
pixels =224
BATCH_SIZE = 32
IMAGE_SIZE = (pixels, pixels)
NUM_CLASSES = 5

I am going to use ResNet model to analyze image classification. To use this model, you need to preprocess. Let me standardize image size to [223, 223, 3], and normalize pixel value tu a [0,1] range. First,let's create variable for hyperparameters.

In [ ]:
datagen_kwargs = dict(rescale = 1./255, validation_split = .20)
dataflow_kwargs = dict(target_size = IMAGE_SIZE, batch_size = BATCH_SIZE, interpolation = "bilinear")

I am going to split dataset into train and validation.  While validation dataset uses for cross validation, the other dataset uses for training model. 

In [ ]:
# Generating batches of tensor image-data and creating validation dataset.
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, 
    **dataflow_kwargs)

Now, do the same for the training data generator. 

In [ ]:
train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, 
    **dataflow_kwargs)

I am going to define class name. To do this, let me map flower class names.

In [ ]:
labels_idx = (train_generator.class_indices)
idx_labels = dict((v,k) for k,v in labels_idx.items())
#Let's take a look how to map classes.
idx_labels

## Building the Model

Datasets are ready to build the model. I am going to train using pretrain model. 

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/4", trainable=False),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', name = 'flower_class') 
])

model.build([None, 224, 224, 3])

## Compiling the Model

After the model is built I am going to compile the model. To do this, let me specify the loss funciton and pick an optimizer.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

## Training the Model

Let's traing the model. First, let me determine the number of batches for training and cross-validation data.

In [ ]:
steps_per_epoch = train_generator.samples // 
train_generator.batch_size
validation_steps = valid_generator.samples // 
valid_generator.batch_size

Now that I am going to fit model.

In [ ]:
model.fit(
    train_generator,
    epochs=5, 
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps)

## Predicting the Data

You can use predict() methodto score these validation images. Let's see predictions for first batch

In [ ]:
sample_test_images, ground_truth_labels = next(valid_generator)
prediction = model.predict(sample_test_images)

731 images and 5 corresponding classes in the cross-validation data print out on the screen. The highest probability in each row represents the prediction. I am going to find position where the highest probability occurs for each row.

In [ ]:
predicted_idx = tf.math.argmax(prediction, axis = -1)

Let me display the result using print command.

In [ ]:
print (predicted_idx)

## Evaluating the Model

To evaluate the model, I am going to use confusion matrix, which compare model output with ground truth. To do this let me convert classes into Pandas Series structure and create variable, which include predictions.

In [ ]:
y_actual = pd.Series(valid_generator.classes)
y_predicted = pd.Series(predicted_idx)

Then I am going to produce the matrix.

In [ ]:
pd.crosstab(y_actual, y_predicted, rownames = ['Actual'], colnames=['Predicted'], margins=True)

Each row represents predicted value and each column display actual values. 
Now that let's take a look statistical report using sklearn library.

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(truth, predicted_results)
print(report)

This results say that the model has the best performance for classification problem.
That is all. In this tutorial. I showed how to use the pretrained model. You can easily train the model according to your analysis usinf the pretrained model in TensorFlow Hub.

## Resources

- [KC Tung, 2021, TensorFlow 2 Pocket Reference](https://www.amazon.com/TensorFlow-Pocket-Reference-Building-Deploying/dp/1492089184)
- [TensorFlow Tutorial](https://www.tensorflow.org/tutorials)

Don't forget to follow on Tirendaz Academy [YouTube-Tr](https://youtube.com/c/tirendazakademi), [YouTube-Eng](https://www.youtube.com/channel/UCFU9Go20p01kC64w-tmFORw), [Twitter](https://twitter.com/TirendazAcademy), [Medium](https://tirendazacademy.medium.com), [GitHub](https://github.com/TirendazAcademy) and [LinkedIn](https://www.linkedin.com/in/tirendaz-academy)